In [1]:
from pyscf import gto,scf
from pyscf.geomopt.berny_solver import optimize
import numpy as np
from alch_deriv import alch_deriv
from FcMole import FcM
import matplotlib.pyplot as plt

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[dL[0][0]])
    dV=mol.intor('int1e_rinv')*dL[1][0]
    for i in range(1,len(dL[0])): 
        mol.set_rinv_orig_(mol.atom_coords()[dL[0][i]])
        dV+=mol.intor('int1e_rinv')*dL[1][i]
    return -dV
#ao_div=mol0.aoslice_by_atom()[:,-2:]
#pt=ao_div[1][0]

In [2]:
benz_atom_opt=""" C   0.000000   1.388317  0.000000;  
   H  -0.000000   2.461627  -0.000000  ;
   C   1.202318   0.694159   0.000000  ;
   H   2.131832   1.230814  -0.000000  ;
   C   1.202318  -0.694159  -0.000000  ;
   H   2.131832  -1.230814  -0.000000  ;
   C   0.000000  -1.388317   0.000000  ;
   H   0.000000  -2.461627   0.000000  ;
   C  -1.202318  -0.694159  -0.000000  ;
   H  -2.131832  -1.230814  -0.000000  ;
   C  -1.202318   0.694159   0.000000  ;
   H  -2.131832   1.230814  -0.000000  
"""
mol_benz = gto.M(atom=benz_atom_opt, basis='6-31G') #angstroms not atomic units
mf_benz=scf.RHF(mol_benz)
e_benz=mf_benz.scf()

converged SCF energy = -230.624474947017


In [3]:
P0=mf_benz.make_rdm1()
print(mol_benz.atom_charges())

[6 1 6 1 6 1 6 1 6 1 6 1]


In [4]:
d1,d2,d3=alch_deriv(mf_benz,dL=[[0,2],[-1,1]])
d1,d2,d3

(2.3639230198983785e-06, -2.4301772918020585, -8.748296366229563e-08)

In [5]:
fmol_benz = FcM(fcs=[[0,2],[-1,1]],atom=benz_atom_opt, basis='6-31G') #angstroms not atomic units
fmf_benz=scf.RHF(fmol_benz)
e_T=fmf_benz.scf()

converged SCF energy = -232.263844927307


In [6]:
e_benz+d1+1/2*d2+1/6*d3+fmol_benz.energy_nuc()-mol_benz.energy_nuc()

-232.2207279745016

In [7]:
#Finite Difference Coefficients
fdc1=np.asarray([1/12,-2/3,0,2/3,-1/12])
fdc2=np.asarray([-1/12,4/3,-5/2,4/3,-1/12])
fdc3=np.asarray([-1/2,1,0,-1,1/2])
fdc4=np.asarray([1,-4,6,-4,1])
coeffs=np.array([fdc1,fdc2,fdc3,fdc4])

In [10]:
dl=.1
Ps=[]
Es=[]
for i in range(-2,3):
    fm=FcM(fcs=[[0,2],[-1*dl*i,1*dl*i]],atom=benz_atom_opt, basis='6-31G')
    mf=scf.RHF(fm)
    Es.append(mf.scf(dm0=mf.init_guess_by_atom()))
    Ps.append(mf.make_rdm1())

converged SCF energy = -230.688387692685
converged SCF energy = -230.640441397141
converged SCF energy = -230.624474947017
converged SCF energy = -230.640441358284
converged SCF energy = -230.68838761513


In [11]:
dV=DeltaV(mol_benz,dL=[[0,2],[-1,1]])

In [12]:
e=e_benz+np.einsum('ij,ij',mf_benz.make_rdm1(),dV) +(fmf_benz.energy_nuc()-mf_benz.energy_nuc())
print("APDFT1",e,np.einsum('ij,ij',mf_benz.make_rdm1(),dV))
for i in range (4):
    dP=np.einsum('xij,x->ij',Ps,coeffs[i]/(dl)**(i+1))
    e+=np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2)
    print("APDFT{}".format(i+2),e,np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2))

APDFT1 -231.00563931402007 2.3639230198983785e-06
APDFT2 -232.22072514356896 -1.2150858295488791
APDFT3 -232.22072538973865 -2.461696933788483e-07
APDFT4 -232.25998254151364 -0.03925715177499508
APDFT5 -232.25997974453887 2.796974761023599e-06


In [13]:
print(e_T)
print(e_benz+d1+1/2*d2+1/6*d3+fmol_benz.energy_nuc()-mol_benz.energy_nuc() ,1/2*d2,1/6*d3)

-232.26384492730736
-232.2207279745016 -1.2150886459010293 -1.4580493943715939e-08


In [16]:
fmol_benz_1 = FcM(fcs=[[0,2],[-.01,.01]],atom=benz_atom_opt, basis='6-31G') #angstroms not atomic units
fmf_benz_1=scf.RHF(fmol_benz_1)
e_T_1=fmf_benz_1.scf()
d1_1,d2_1,d3_1=alch_deriv(fmf_benz_1,dL=[[0,2],[-1,1]])
fmol_benz_2 = FcM(fcs=[[0,2],[.01,-.01]],atom=benz_atom_opt, basis='6-31G') #angstroms not atomic units
fmf_benz_2=scf.RHF(fmol_benz_2)
e_T_2=fmf_benz_2.scf()
d1_2,d2_2,d3_2=alch_deriv(fmf_benz_2,dL=[[0,2],[-1,1]])

converged SCF energy = -230.624634570724
converged SCF energy = -230.624634574613


In [17]:
d4=(d3_1-d3_2)/.02
d5=(d3_1-2*d3+d3_2)/.01**2
print(d4/24,d5/120)

-0.03893047503380307 1.095462354942095e-06


In [18]:
print(e_T)
print(e_benz+d1+1/2*d2+1/6*d3+d4/24+d5/120+fmol_benz.energy_nuc()-mol_benz.energy_nuc() ,1/2*d2,1/6*d3)

-232.26384492730736
-232.25965735407306 -1.2150886459010293 -1.4580493943715939e-08


In [19]:
d3s=[]
for i in range(-2,3):
    fm=FcM(fcs=[[0,2],[-1*dl*i,1*dl*i]],atom=benz_atom_opt, basis='6-31G')
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_atom())
    d3s.append(alch_deriv(mf,dL=[[0,2],[-1,1]])[2])

converged SCF energy = -230.688387692685
converged SCF energy = -230.640441397141
converged SCF energy = -230.624474947017
converged SCF energy = -230.640441358285
converged SCF energy = -230.688387615129


In [24]:
d3s=np.asarray(d3s)
for i in range (4):
    print("APDFT{} contr.".format(i+4),np.sum(d3s*coeffs[i])/(dl)**(i+1)/np.math.factorial(i+4))

APDFT4 contr. -0.03894617749615839
APDFT5 contr. 1.3860221084275681e-08
APDFT6 contr. -0.004134841429459715
APDFT7 contr. -5.18123884294855e-08


In [25]:
-232.25965735407306-0.004134841429459715,e_T

(-232.2637921955025, -232.26384492730736)